# Content Creator Agent with Toolhouse.ai


# Overview
Build a content creation agent that can generate blog posts, social media content, and marketing materials using Toolhouse.ai's research and generation capabilities.


In [1]:
# Install required packages
!pip install toolhouse openai python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.0/264.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 4.1 MB/s eta 0:00:00


In [2]:
# Import dependencies
import os
from dotenv import load_dotenv
from openai import OpenAI
from toolhouse import Toolhouse, Provider
from datetime import datetime

# Load environment variables
load_dotenv()

False

In [ ]:
# Set up API keys
TOOLHOUSE_API_KEY = "toolhouse-api-key-here"
OPENAI_API_KEY = "openai-api-key-here"


# Initialize clients
client = OpenAI(api_key=OPENAI_API_KEY)
th = Toolhouse(api_key=TOOLHOUSE_API_KEY, provider=Provider.OPENAI)

# Set timezone (use numeric value)
th.set_metadata('timezone', -5)  # EST timezone

# Model configuration
MODEL = 'gpt-4o'

In [5]:
def create_blog_post(topic, target_audience="general", word_count=800, tone="professional"):
    """
    Create a blog post with research-backed content

    Args:
        topic: The blog post topic
        target_audience: Target audience description
        word_count: Desired word count
        tone: Writing tone (professional, casual, technical, etc.)
    """

    system_prompt = f"""
    You are an expert content creator and blogger. Create a comprehensive blog post on "{topic}".

    Requirements:
    - Target audience: {target_audience}
    - Tone: {tone}
    - Word count: approximately {word_count} words
    - Include current information and statistics
    - Add relevant examples and case studies
    - Structure with clear headings and subheadings
    - Include a compelling introduction and conclusion
    - Add actionable insights or takeaways

    Process:
    1. Research the topic for current information
    2. Find relevant statistics and examples
    3. Create an outline
    4. Write the full blog post
    5. Include relevant links and sources

    Use available tools to research current information and trends.
    """

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Create a blog post about: {topic}"}
    ]

    # Get response with tools
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=th.get_tools(),
        max_tokens=2500
    )

    # Process tools and continue
    messages = th.run_tools(response, messages)

    # Check if we have valid messages
    if messages and len(messages) > 0:
        # Get final blog post
        final_response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            max_tokens=2500
        )
        return final_response.choices[0].message.content
    else:
        # Return original response if tool processing failed
        return response.choices[0].message.content

In [6]:
# Example usage
blog_post = create_blog_post(
    topic="The Future of Remote Work in 2025",
    target_audience="business professionals and team leaders",
    word_count=1000,
    tone="professional yet engaging"
)
print(blog_post)

## Future of Remote Work in 2025

### Key Predictions and Trends

1. **Hybrid Models Becoming Norm**: By 2025, the hybrid work model will become standardized, with clear guidelines for balancing remote and office work. Many companies will adopt this model to offer flexibility and improve collaboration.

2. **Technological Advancements**: AI and automation will play a more significant role, with tools to enhance productivity and communication in remote settings. AI-driven project management and real-time translation in virtual meetings are expected to become more commonplace.

3. **VR and AR in Collaboration**: Virtual and augmented reality will redefine how remote teams collaborate, creating immersive environments for effective virtual meetings and training sessions.

4. **Focus on Employee Wellbeing**: Companies will prioritize tracking employee wellbeing using technology, leading to personalized wellbeing strategies and flexible work schedules to prevent burnout.

5. **Diversity and 

In [7]:
class SocialMediaCreator:
    def __init__(self, client, toolhouse):
        self.client = client
        self.th = toolhouse

        # Platform-specific requirements
        self.platform_specs = {
            "twitter": {"max_chars": 280, "hashtags": 2, "tone": "concise"},
            "linkedin": {"max_chars": 3000, "hashtags": 5, "tone": "professional"},
            "instagram": {"max_chars": 2200, "hashtags": 10, "tone": "engaging"},
            "facebook": {"max_chars": 2000, "hashtags": 3, "tone": "conversational"},
            "tiktok": {"max_chars": 1000, "hashtags": 5, "tone": "casual"}
        }

    def create_social_post(self, topic, platform, brand_voice="friendly", include_cta=True):
        """Create platform-optimized social media content"""

        specs = self.platform_specs.get(platform, {"max_chars": 1000, "hashtags": 3, "tone": "neutral"})

        system_prompt = f"""
        Create a {platform} post about "{topic}".

        Platform Requirements:
        - Maximum characters: {specs['max_chars']}
        - Suggested hashtags: {specs['hashtags']}
        - Platform tone: {specs['tone']}
        - Brand voice: {brand_voice}
        - Include CTA: {include_cta}

        Guidelines:
        1. Research current trends related to the topic
        2. Create engaging, platform-appropriate content
        3. Include relevant and trending hashtags
        4. Add a clear call-to-action if requested
        5. Use emojis appropriately for the platform
        6. Ensure content fits character limits

        Use tools to research trending topics and hashtags.
        """

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Create a {platform} post about: {topic}"}
        ]

        response = self.client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=self.th.get_tools()
        )

        messages = self.th.run_tools(response, messages)

        if messages and len(messages) > 0:
            final_response = self.client.chat.completions.create(
                model=MODEL,
                messages=messages
            )
            return final_response.choices[0].message.content
        else:
            return response.choices[0].message.content

    def create_content_series(self, main_topic, platforms, num_posts=5):
        """Create a series of related posts across platforms"""

        system_prompt = f"""
        Create a series of {num_posts} related social media posts about "{main_topic}".

        Requirements:
        - Each post should cover a different aspect of the main topic
        - Ensure variety in content types (tips, questions, facts, stories)
        - Maintain consistency in messaging across the series
        - Research current trends and incorporate relevant information
        - Create engaging content that encourages interaction

        Platforms to create for: {', '.join(platforms)}
        """

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Create a content series about: {main_topic}"}
        ]

        response = self.client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=self.th.get_tools()
        )

        messages = self.th.run_tools(response, messages)

        if messages and len(messages) > 0:
            final_response = self.client.chat.completions.create(
                model=MODEL,
                messages=messages,
                max_tokens=2000
            )
            return final_response.choices[0].message.content
        else:
            return response.choices[0].message.content

In [8]:
# Example usage
social_creator = SocialMediaCreator(client, th)

# Single platform post
twitter_post = social_creator.create_social_post(
    topic="AI productivity tools",
    platform="twitter",
    brand_voice="tech-savvy",
    include_cta=True
)
print("Twitter Post:", twitter_post)

Twitter Post: Here is a summary of current trends and insights regarding AI productivity tools based on web search results and Google Trends analysis:

1. **Popular AI Productivity Tools (2024-2025)**:
   - **Diversity in Functionality**: AI productivity tools cover a wide range of functions, including task management, scheduling, email management, learning, sales management, language translation, meeting summarization, research assistance, content creation, and more. Notable tools include:
     - **Language and Learning**: Learnt AI, Caktus AI, and LanguaTalk.
     - **Task and Project Management**: Goblin.tools, Notion AI, Taskade.
     - **Communication and Collaboration**: Superhuman for email management, Reclaim.ai for scheduling.
     - **Creative and Content Tools**: Synthesia for video creation, Copy AI for copywriting, Canva for design.
   - **Cross-Sector Use**: These tools are prevalent across education, business, marketing, and technology sectors.

2. **Market Expansion**:


In [9]:
content_series = social_creator.create_content_series(
    main_topic="Sustainable business practices",
    platforms=["linkedin", "twitter", "instagram"],
    num_posts=3
)
print("\nContent Series:", content_series)


Content Series: ### Current Trends in Sustainable Business Practices

1. **Circular Economy**: This trend focuses on reducing waste and pollution through sustainable practices such as designing durable and recyclable products, using materials efficiently, and offering product-as-a-service models.

2. **Social Impact**: Companies are assessing the societal effects of their operations and considering social impact when making business decisions.

3. **Digital Transformation**: Leveraging technology for sustainability efforts, businesses are utilizing digital tools to improve efficiency, track sustainability metrics, and reduce their environmental footprint.

4. **Proactive Solutions**: Businesses are shifting towards Sustainability 3.0, where they actively contribute to solving global sustainability challenges, rather than just mitigating negative impacts.

5. **Consumer and Employee Preferences**: A significant consumer preference for sustainable brands (80%) is influencing business st

In [10]:
def create_email_campaign(product_service, target_audience, campaign_goal, email_type="promotional"):
    """
    Create email marketing campaign content

    Args:
        product_service: What you're promoting
        target_audience: Description of target audience
        campaign_goal: Primary goal (sales, awareness, retention, etc.)
        email_type: Type of email (promotional, newsletter, welcome, etc.)
    """

    system_prompt = f"""
    Create an email marketing campaign for "{product_service}".

    Campaign Details:
    - Target audience: {target_audience}
    - Primary goal: {campaign_goal}
    - Email type: {email_type}

    Create:
    1. Compelling subject line (test A/B variations)
    2. Email content with clear structure:
       - Attention-grabbing opening
       - Value proposition
       - Benefits and features
       - Social proof or testimonials
       - Clear call-to-action
       - Professional closing
    3. Preheader text
    4. Alternative versions for A/B testing

    Research:
    - Current email marketing trends
    - Best practices for the target audience
    - Competitive analysis if relevant

    Make the content persuasive, engaging, and action-oriented.
    """

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Create email campaign for: {product_service}"}
    ]

    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=th.get_tools(),
        max_tokens=2000
    )

    messages = th.run_tools(response, messages)

    if messages and len(messages) > 0:
        final_response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            max_tokens=2000
        )
        return final_response.choices[0].message.content
    else:
        return response.choices[0].message.content

In [11]:
email_campaign = create_email_campaign(
    product_service="AI-powered project management software",
    target_audience="small business owners and team leaders",
    campaign_goal="increase trial signups",
    email_type="promotional"
)
print(email_campaign)

For small business owners looking to enhance their email marketing and project management strategies, here are some key insights based on recent trends and best practices:

### Email Marketing Trends 2023
1. **Personalization and AI**: Utilize AI-driven tools for personalized content, optimizing send times, and creating tailored experiences for your subscribers. Treat your audience as VIPs with customized messages beyond just name personalization.

2. **User-Generated Content**: Incorporate testimonials, reviews, and photos from your customers to create authentic and engaging campaigns.

3. **Mobile Optimization**: Ensure your emails are responsive and interactive by using designs optimized for mobile devices, including AMP technology for dynamic content.

4. **Data Privacy**: Prioritize consumer privacy by complying with regulations like GDPR and CCPA, as well as allowing easy data preference changes and unsubscribing options.

5. **Interactive Content**: Engage your audience with qui

In [12]:
class SEOContentCreator:
    def __init__(self, client, toolhouse):
        self.client = client
        self.th = toolhouse

    def research_keywords(self, main_topic, content_type="blog"):
        """Research SEO keywords for content"""

        system_prompt = f"""
        Research SEO keywords for "{main_topic}" for {content_type} content.

        Tasks:
        1. Search for trending keywords related to the topic
        2. Identify long-tail keywords with good potential
        3. Find related questions people are asking
        4. Analyze search intent behind keywords
        5. Suggest primary and secondary keywords

        Return a structured list of:
        - Primary keyword (1-2)
        - Secondary keywords (3-5)
        - Long-tail keywords (5-10)
        - Related questions/topics
        - Search volume estimates if available

        Use web search tools to find current keyword trends.
        """

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Research keywords for: {main_topic}"}
        ]

        response = self.client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=self.th.get_tools()
        )

        messages = self.th.run_tools(response, messages)

        if messages and len(messages) > 0:
            final_response = self.client.chat.completions.create(
                model=MODEL,
                messages=messages
            )
            return final_response.choices[0].message.content
        else:
            return response.choices[0].message.content

    def create_seo_content(self, topic, keywords, content_type="blog", word_count=1200):
        """Create SEO-optimized content"""

        system_prompt = f"""
        Create SEO-optimized {content_type} content about "{topic}".

        SEO Requirements:
        - Target keywords: {keywords}
        - Word count: approximately {word_count} words
        - Include keywords naturally (avoid keyword stuffing)
        - Create compelling meta title and description
        - Use proper heading structure (H1, H2, H3)
        - Include internal linking opportunities
        - Add FAQ section if appropriate
        - Optimize for featured snippets
        - Include semantic keywords

        Content Structure:
        1. SEO-optimized title (H1)
        2. Engaging introduction with target keyword
        3. Well-structured body with subheadings
        4. Conclusion with call-to-action
        5. Meta description
        6. Suggested internal links

        Research current information and include up-to-date facts and statistics.
        """

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Create SEO content for: {topic}"}
        ]

        response = self.client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=self.th.get_tools(),
            max_tokens=2500
        )

        messages = self.th.run_tools(response, messages)

        if messages and len(messages) > 0:
            final_response = self.client.chat.completions.create(
                model=MODEL,
                messages=messages,
                max_tokens=2500
            )
            return final_response.choices[0].message.content
        else:
            return response.choices[0].message.content

    def create_content_cluster(self, main_topic, num_articles=5):
        """Create a content cluster around a main topic"""

        system_prompt = f"""
        Create a content cluster strategy for "{main_topic}".

        Requirements:
        1. Research the main topic and identify subtopics
        2. Create {num_articles} article ideas that support the main topic
        3. Ensure proper internal linking structure
        4. Plan keyword targeting for each article
        5. Create topic clusters that boost overall SEO

        For each article in the cluster:
        - Provide title and brief description
        - List target keywords
        - Suggest internal linking strategy
        - Estimate content length

        Use search tools to research current trends and competition.
        """

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Create content cluster for: {main_topic}"}
        ]

        response = self.client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=self.th.get_tools()
        )

        messages = self.th.run_tools(response, messages)

        if messages and len(messages) > 0:
            final_response = self.client.chat.completions.create(
                model=MODEL,
                messages=messages,
                max_tokens=1500
            )
            return final_response.choices[0].message.content
        else:
            return response.choices[0].message.content

In [13]:
# Example usage
seo_creator = SEOContentCreator(client, th)

# Research keywords
keywords = seo_creator.research_keywords("machine learning for beginners")
print("Keywords Research:", keywords)

# Create SEO content
seo_content = seo_creator.create_seo_content(
    topic="Machine Learning for Beginners",
    keywords="machine learning, beginner guide, ML tutorial, artificial intelligence basics",
    content_type="blog",
    word_count=1500
)
print("\nSEO Content:", seo_content[:500] + "...")

# Create content cluster
content_cluster = seo_creator.create_content_cluster("Machine Learning", num_articles=4)
print("\nContent Cluster:", content_cluster)

Keywords Research: Here's a compilation of information regarding the trending keywords, long-tail keywords, related questions, and search intent for "machine learning for beginners":

### Trending Keywords for "Machine Learning for Beginners":
1. Python programming
2. Machine learning libraries (e.g., TensorFlow, Scikit-learn)
3. Supervised learning
4. Unsupervised learning
5. Reinforcement learning
6. Data sets
7. Beginner-friendly machine learning projects (e.g., predicting weather patterns, building recommendation systems)

### Long-Tail Keywords:
1. Machine learning for absolute beginners
2. Introduction to machine learning for dummies
3. Easy machine learning tutorials
4. Machine learning basics for beginners with examples
5. Reinforcement learning for beginners
6. Machine learning for self-driving cars tutorial
7. Beginner's guide to AI in self-driving vehicles

### Questions People Are Asking:
1. What is machine learning?
2. How do I start learning machine learning?
3. What prog

In [14]:
def create_video_script(topic, video_type="educational", duration_minutes=5, platform="youtube"):
    """
    Create video scripts for different platforms

    Args:
        topic: Video topic
        video_type: Type of video (educational, promotional, entertainment)
        duration_minutes: Approximate video length
        platform: Target platform (youtube, tiktok, instagram, etc.)
    """

    platform_specs = {
        "youtube": {"style": "detailed", "hook_duration": "first 15 seconds"},
        "tiktok": {"style": "fast-paced", "hook_duration": "first 3 seconds"},
        "instagram": {"style": "visually-focused", "hook_duration": "first 5 seconds"},
        "facebook": {"style": "conversational", "hook_duration": "first 10 seconds"}
    }

    specs = platform_specs.get(platform, {"style": "standard", "hook_duration": "first 10 seconds"})

    system_prompt = f"""
    Create a video script about "{topic}" for {platform}.

    Video Specifications:
    - Type: {video_type}
    - Duration: {duration_minutes} minutes
    - Platform: {platform}
    - Style: {specs['style']}
    - Hook timing: {specs['hook_duration']}

    Script Structure:
    1. Strong hook/opening ({specs['hook_duration']})
    2. Introduction and context
    3. Main content points (3-5 key points)
    4. Engaging transitions between sections
    5. Visual cues and suggestions
    6. Call-to-action and conclusion
    7. Suggested thumbnail ideas

    Include:
    - Timestamps for key sections
    - Visual descriptions
    - Tone and pacing notes
    - Engagement prompts (likes, comments, shares)

    Research current trends for the topic and platform to ensure relevance.
    """

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Create video script for: {topic}"}
    ]

    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=th.get_tools(),
        max_tokens=2000
    )

    messages = th.run_tools(response, messages)

    if messages and len(messages) > 0:
        final_response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            max_tokens=2000
        )
        return final_response.choices[0].message.content
    else:
        return response.choices[0].message.content

# Example usage
video_script = create_video_script(
    topic="5 Productivity Hacks for Remote Workers",
    video_type="educational",
    duration_minutes=3,
    platform="youtube"
)
print(video_script)

Here are five productivity hacks for remote workers, focusing on 2023 trends and strategies:

1. **Optimize Technology Use**: Tools like Slack, Zoom, and Microsoft Teams are essential for remote communication. Using them efficiently to maintain clear, open lines of communication can help keep remote teams productive. For task management and time tracking, tools such as Asana, Trello, and TrackingTime's cloud-based platforms are recommended [Remote Work Productivity: Navigating Trends and Effective Strategies](https://trackingtime.co/remote-work/remote-work-productivity.html).

2. **Encourage Work-Life Balance**: Remote work often blurs the lines between personal and professional life, leading to burnout. Encouraging regular breaks, setting clear work boundaries, and using tools like Reclaim AI that help schedule personal activities can promote better work-life integration [Top 20 AI Productivity Tools for Remote Workers](https://wrkland.com/blog/top-20-ai-productivity-tools-for-remote-

In [15]:
class ContentRepurposer:
    def __init__(self, client, toolhouse):
        self.client = client
        self.th = toolhouse

    def analyze_content(self, original_content, content_type="blog"):
        """Analyze content for repurposing opportunities"""

        system_prompt = f"""
        Analyze this {content_type} content for repurposing opportunities.

        Tasks:
        1. Extract key points and themes
        2. Identify quotable snippets
        3. Find statistical data or facts
        4. Locate actionable tips or advice
        5. Identify visual content opportunities
        6. Suggest different content formats

        Content to analyze:
        {original_content[:2000]}...

        Provide a structured analysis with specific suggestions for repurposing.
        """

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "Analyze this content for repurposing"}
        ]

        response = self.client.chat.completions.create(
            model=MODEL,
            messages=messages
        )

        return response.choices[0].message.content

    def repurpose_content(self, original_content, target_format, platform=None):
        """Repurpose content into different formats"""

        format_instructions = {
            "social_posts": "Create 5-7 social media posts highlighting key points",
            "infographic": "Create infographic content with key stats and visuals",
            "email_newsletter": "Transform into email newsletter format",
            "podcast_outline": "Create detailed podcast episode outline",
            "video_script": "Convert to engaging video script",
            "twitter_thread": "Create comprehensive Twitter thread",
            "linkedin_article": "Adapt for LinkedIn article format",
            "presentation": "Create slide-by-slide presentation outline"
        }

        instruction = format_instructions.get(target_format, f"Convert to {target_format} format")

        system_prompt = f"""
        Repurpose the following content into {target_format} format.

        Instructions: {instruction}
        {f"Platform: {platform}" if platform else ""}

        Requirements:
        - Maintain core message and value
        - Adapt tone and style for the new format
        - Optimize for the target platform/medium
        - Include engaging elements appropriate for the format
        - Preserve key statistics and facts
        - Add relevant calls-to-action

        Original content:
        {original_content[:2000]}...

        Create the repurposed content following best practices for {target_format}.
        """

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Repurpose into {target_format}"}
        ]

        response = self.client.chat.completions.create(
            model=MODEL,
            messages=messages,
            max_tokens=2000
        )

        return response.choices[0].message.content

    def create_content_calendar(self, base_content, platforms, duration_weeks=4):
        """Create a content calendar from base content"""

        system_prompt = f"""
        Create a {duration_weeks}-week content calendar repurposing this base content across multiple platforms.

        Platforms: {', '.join(platforms)}

        Requirements:
        1. Distribute content strategically across {duration_weeks} weeks
        2. Create platform-specific versions
        3. Vary content types (posts, stories, videos, etc.)
        4. Include optimal posting times
        5. Plan content themes for each week
        6. Ensure cohesive messaging across platforms

        Base content:
        {base_content[:1500]}...

        Create a detailed calendar with specific posts for each platform and day.
        """

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "Create content calendar"}
        ]

        response = self.client.chat.completions.create(
            model=MODEL,
            messages=messages,
            max_tokens=2000
        )

        return response.choices[0].message.content

In [16]:
# Example usage
repurposer = ContentRepurposer(client, th)

# Sample blog content (you would use your actual content here)
sample_blog = """
The Future of Remote Work: 10 Trends Shaping 2025

Remote work has fundamentally transformed how we approach professional life...
[Your full blog content here]
"""

# Analyze content
analysis = repurposer.analyze_content(sample_blog, "blog")
print("Content Analysis:", analysis)

# Repurpose into different formats
social_posts = repurposer.repurpose_content(sample_blog, "social_posts", "linkedin")
print("\nSocial Posts:", social_posts)

twitter_thread = repurposer.repurpose_content(sample_blog, "twitter_thread")
print("\nTwitter Thread:", twitter_thread)

# Create content calendar
content_calendar = repurposer.create_content_calendar(
    sample_blog,
    platforms=["twitter", "linkedin", "instagram"],
    duration_weeks=3
)
print("\nContent Calendar:", content_calendar)

Content Analysis: Since I don't have the actual text from the blog "The Future of Remote Work: 10 Trends Shaping 2025," I'll provide a general framework for how you might analyze similar content for repurposing opportunities. 

### 1. Extract Key Points and Themes
- **Trends:** Highlight any specific trends mentioned such as the increase in hybrid work models, reliance on digital collaboration tools, or changes in corporate culture.
- **Future Predictions:** Capture predictions about how remote work will evolve by 2025, such as changes in legal frameworks or the integration of AI in workflow management.
- **Challenges and Solutions:** Note discussions about remote work challenges, such as communication barriers, and corresponding solutions like enhanced digital meeting platforms.

### 2. Identify Quotable Snippets
- Extract key sentences that encapsulate the vision or insights discussed, such as "Remote work is here to stay, and adaptability is the keyword for businesses aiming to thri

In [17]:
def review_content_quality(content, content_type="blog", brand_guidelines=None):
    """Review content for quality, tone, and brand alignment"""

    system_prompt = f"""
    Review this {content_type} content for quality and brand alignment.

    Evaluation Criteria:
    1. Clarity and readability
    2. Grammar and spelling accuracy
    3. Brand voice consistency
    4. SEO optimization (if applicable)
    5. Engagement potential
    6. Factual accuracy
    7. Call-to-action effectiveness
    8. Overall structure and flow

    {f"Brand Guidelines: {brand_guidelines}" if brand_guidelines else ""}

    Content to review:
    {content[:2000]}...

    Provide:
    1. Overall quality score (1-10)
    2. Specific areas for improvement
    3. Suggestions for enhancement
    4. Compliance with brand guidelines
    5. Recommended edits or revisions

    Use fact-checking tools if needed to verify claims.
    """

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "Review this content"}
    ]

    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=th.get_tools(),
        max_tokens=1500
    )

    messages = th.run_tools(response, messages)

    if messages and len(messages) > 0:
        final_response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            max_tokens=1500
        )
        return final_response.choices[0].message.content
    else:
        return response.choices[0].message.content

In [18]:
# Example usage
quality_review = review_content_quality(
    content=blog_post,
    content_type="blog",
    brand_guidelines="Professional yet approachable tone, focus on practical value, include data-driven insights"
)
print("Quality Review:", quality_review)

Quality Review: To provide a thorough review of the blog content titled "Future of Remote Work in 2025," let's evaluate it based on the specified criteria:

### Overall Quality Score: 8/10

### Specific Areas for Improvement:
1. **Clarity and Readability**: While the content is well-organized and succinct, adding subheadings under each trend might improve navigability and readability.
2. **Grammar and Spelling Accuracy**: Overall, the grammar and spelling are accurate, but minor edits can further refine the text.
3. **Brand Voice Consistency**: The content maintains a professional yet approachable tone, aligning well with the brand guidelines.
4. **SEO Optimization**: Although keywords are naturally integrated, further refinement using specific key phrases related to remote work trends may enhance SEO.
5. **Engagement Potential**: Including interactive elements such as polls or questions might increase reader engagement.
6. **Factual Accuracy**: A quick fact-check is recommended to ver